In [1]:
import numpy as np
import pyarrow.parquet as pq
import cv2

In [2]:
#Cellule destinée à extraire les images du fichier parquet

index = 0
table = pq.read_table('test-00000-of-00001.parquet') #changer le nom du fichier selon les besoins
for img in table['page_image']:
    b = img['bytes'].as_py()
    with open(f'{index}.png', 'wb') as f:
        f.write(b)
        index+=1


In [3]:
#Cellule destinée à extraire les BBs du fichier parquet

index=0
for arr in table['zone_polygons']:
    txt=arr.as_py()
    with open(f'{index}.txt', 'w') as f:
        f.write(str(txt))
        index+=1

In [4]:
#Cellule destinée à extraire les classes des BBs du fichier parquet

index=0

for i in table['zone_classes']:
    text= i.as_py()
    with open(f'classes{index}.txt','w') as f:
        for z in text:
            f.write(str(z))
        index+=1

In [5]:
def remove_char_at_position(s, pos):
    if pos < 0 or pos >= len(s):
        return s
    return s[:pos] + s[pos+1:]

def from_text_to_list(nombre):
    with open (f"{nombre}.txt",'r') as f:
        string= f.read().replace("\n", "")
        string= remove_char_at_position(string, 0)
        string= remove_char_at_position(string, 0)
        liste= string.split('], [')
    return liste





In [6]:
def appliquer_coord(liste, nombre, liste_couleurs):
    liste2=[]
    for i in range(0, len(liste)):
        valeur= liste[i]
        valeur2 = valeur.replace('[','').replace(']','')
        liste2.append(valeur2)
    for i in range(len(liste2)-1, 3, -1):
            if (i - 4) % 5 == 0:
                del liste2[i]
    im= cv2.imread(f'{nombre}.png')
    f = open(f'classes{nombre}.txt', 'r')
    contenu = str(f.read())
    height, width = im.shape[:2]
    thickness = 5
    setlist=[]
    
    for z in range(1, len(liste2), 2):
        
       
        
        listes_coord= str(liste2[z]).split(',')
        hauteur = round(float((listes_coord[1]))*height/100)
        largeur = round((float(listes_coord[0]))*width/100)
        set_coord = (largeur, hauteur)
        setlist.append(set_coord)
    index =0    
    for i in range(0, len(setlist), 2):
        try :
            couleur = liste_couleurs[int(contenu[index])]
        except:
            couleur = liste_couleurs[0]
        im=cv2.rectangle(im, setlist[i+1],setlist[i], color=couleur, thickness=thickness)
        index+=1
    cv2.imwrite(f"{nombre}v2.png", im)
    
    

In [7]:
def from_text_to_annotated_image (nombre, liste_couleurs):
    liste = from_text_to_list(nombre)
    appliquer_coord(liste, nombre, liste_couleurs)
   
    return "Terminé"

In [8]:
liste_couleurs = [(0,0,0), (0,225,0), (225,0,0), (0,0,225), (120, 120, 0), (10, 50,120),(60,120,10), (0,225, 120), (120,225,0), (225,225,225)] #la liste de couleurs peut être modifié selon vos besoins
for i in range (0,48): #remplacer le deuxième nombre par le nombre d'images +1
    from_text_to_annotated_image(i, liste_couleurs)